In [7]:
import json
import os
import pandas as pd
import requests
import numpy as np
from IPython.display import display_html
from itertools import chain,cycle
import plotly.offline as py_offline
import plotly.graph_objects as go

pd.options.mode.chained_assignment = None  # default='warn'

class GDKPCalcuator:
    
    def __init__(self, warcraftLogURL, pot, management = 0.15, tankCut = 0.1375, healerCut = 0.2665, noOfTanks = 3, noOfHealers = 6,
                 namesToRemove = [], bonusCutsDict = {}, deductionsCutsDict = {}):
                  
        self.noOfTanks = noOfTanks
        self.noOfHealers = noOfHealers
        self.pot = pot
        self.bonusCutsDict = bonusCutsDict
        self.deductionsCutsDict = deductionsCutsDict
        self.namesToRemove = namesToRemove
        self.supportsCutDict = {'Druid-Balance': 0.0125, 'Priest-Shadow' : 0.0175, 'Shaman-Enhancement' : 0.0125, \
                                'Shaman-Elemental' : 0.0125, 'Hunter-Survival' : 0.01, 'Paladin-Retribution' : 0.0125}
        
        self.warcraftLogURL = warcraftLogURL
        self.df = self.__generateBaseDF(self.warcraftLogURL)
        
            
        self.tanks = list(self.df[self.df.Role == "Tank"].index)
        self.healers = list(self.df[self.df.Role == "Healer"].index)
        self.dps = list(self.df[self.df.Role == "DPS"].index)    
        
        self.managementCut = 0.15 * self.pot
        self.raiderPot = self.pot - self.managementCut
        self.tankCutTotal = tankCut * self.raiderPot
        self.healerCutTotal = healerCut * self.raiderPot
        self.supportCutTotal = self.__getSupportCutTotal(self.df)
        self.bonusCutTotal = self.__getBonusCutTotal(self.df)
        self.dpsCutTotal = self.raiderPot - self.tankCutTotal - self.healerCutTotal - self.supportCutTotal - self.bonusCutTotal
        
        self.df = self.__addCuts(self.df)
        
    def removePlayers(self, df):
        
        return df.drop(self.namesToRemove)
            
            
    def diagnostics(self):
    
        print("Tank Cut    : ", str(round(self.tankCutTotal, 3)), "\nTank Cut DF : ", str(round(self.df["Tank Cut"].sum(), 3)) + "\n\n")
        print("Healer Cut    : ", str(round(self.healerCutTotal, 3)), "\nHealer Cut DF : ", str(round(self.df["Healer Cut"].sum(), 3)) + "\n\n")
        print("DPS Cut    : ", str(round(self.dpsCutTotal, 3)), "\nDPS Cut DF : ", str(round(self.df["Damage Cut"].sum(), 3)) + "\n\n")
        print("Support Cut    : ", str(round(self.supportCutTotal, 3)), "\nSupport Cut DF : ", str(round(self.df["Support Cut"].sum(), 3)) + "\n\n")
        print("Bonus Cut    : ", str(round(self.bonusCutTotal, 3)), "\nBonus Cut DF : ", str(round(self.df["Bonus Cut"].sum(), 3)) + "\n\n")
        
        
        print("\nTotal Pot      : ", round(self.pot, 3))
        print("Management Cut : ", round(self.managementCut, 3))
        print("Raider Cut     : ", round(self.raiderPot, 3))
        
        print("\nTank Cut    : ", round(self.tankCutTotal, 3))
        print("Healer Cut  : ", round(self.healerCutTotal, 3))
        print("DPS Cut     : ", round(self.dpsCutTotal, 3))
        print("Support Cut : ", round(self.supportCutTotal, 3))
        print("Bonus Cut   : ", round(self.bonusCutTotal, 3))
        print("Total Cuts  : ", round(self.tankCutTotal + self.healerCutTotal + self.dpsCutTotal + self.supportCutTotal + self.bonusCutTotal, 3) )
        
    def __getSupportCutTotal(self, df):
        
        supportCutPercentage = 0
        for index, row in self.df[self.df.Spec.isin(list(self.supportsCutDict.keys()))].iterrows():
            supportCutPercentage += self.supportsCutDict[row.Spec]
            
        
        return self.raiderPot * supportCutPercentage
     
    def __getBonusCutTotal(self, df):
        
        bonusCutPercentage = 0
        for player, amount in self.bonusCutsDict.items():
            bonusCutPercentage += amount
        
        return self.raiderPot * bonusCutPercentage
        
    def __generateBaseDF(self, warcraftLogURL): #https://tbc.warcraftlogs.com/reports/1yq4bAzZ7JwYMvDH
        
        baseUrl = "https://classic.warcraftlogs.com:443/v1/"
        logId = warcraftLogURL.split("/")[-1]
        apiKey = "16b730dd4be606d61ce2920212baa71c"
        apiKeyString = "?translate=true&api_key=" + apiKey;


        playersURL = baseUrl + "report/tables/casts/" + logId + apiKeyString + "&start=0&end=999999999999"
        damageURL = baseUrl + "report/tables/damage-done/" + logId + apiKeyString + "&start=0&end=999999999999";
        healingURL = baseUrl + "report/tables/healing/" + logId + apiKeyString + "&start=0&end=999999999999";
        damageTakenURL = baseUrl + "report/tables/damage-taken/" + logId + apiKeyString + "&start=0&end=999999999999";

        playersLog = requests.get(playersURL)
        damageLog = requests.get(damageURL)
        healingLog = requests.get(healingURL)
        damagaTakenLog = requests.get(damageTakenURL)

        dataPlayers = playersLog.json()
        dataDamage = damageLog.json()
        datahealing = healingLog.json()
        dataDmgTaken = damagaTakenLog.json()

        desiredColumnsDataPlayers = ["name", "type", "icon"]
        desiredColumnsDataDamage = ["name", "total"]
        desiredColumnsDatahealing = ["name", "total"]
        desiredColumnsDataDmgTaken = ["name", "total"]
        
        df = pd.DataFrame(dataPlayers['entries'])[desiredColumnsDataPlayers]\
        .rename(columns={"name": "Name", "type": "Class", 'icon' : "Spec"}).set_index('Name')

        tempDF = pd.DataFrame(dataDamage['entries'])[desiredColumnsDataDamage]\
        .rename(columns={"name": "Name", "total": "Damage Done"}).set_index('Name')

        df = df.join(tempDF, how='outer')

        tempDF = pd.DataFrame(datahealing['entries'])[desiredColumnsDatahealing]\
        .rename(columns={"name": "Name", "total": "Healing Done"}).set_index('Name')

        df = df.join(tempDF, how='inner')

        tempDF = pd.DataFrame(dataDmgTaken['entries'])[desiredColumnsDataDmgTaken]\
        .rename(columns={"name": "Name", "total": "Damage Taken"}).set_index('Name')

        df = df.join(tempDF, how='inner')
        
        if self.namesToRemove != None:
            df = self.removePlayers(df)
            
        tanks = list(df.sort_values('Damage Taken', ascending=False).head(self.noOfTanks).index)
        healers = list(df.sort_values('Healing Done', ascending=False).head(self.noOfHealers).index)

        roles = {}

        for player in tanks:
            roles[player] = "Tank"
        for player in healers:
            roles[player] = "Healer"

        df["Role"] = pd.Series(roles)
        df["Role"] = df["Role"].fillna("DPS")

        return df       
             
    def __addCuts(self, df): #https://tbc.warcraftlogs.com/reports/1yq4bAzZ7JwYMvDH
        
        df = self.__addDamageCuts(df)
        df = self.__addTankCuts(df)
        df = self.__addHealerCuts(df)
        df = self.__addSupportCuts(df)
        df = self.__addBonusCuts(df)
        df = self.__addDeductions(df)
        
        df["Total Cut"] = df["Damage Cut"] + df["Support Cut"] + df["Tank Cut"] + df["Healer Cut"] + df["Bonus Cut"] + df["Deductions"]
        df["Total Cut"] = np.ceil(df["Total Cut"])
        df["Total Cut"] = df["Total Cut"].astype('int')
        
        return df
    
    def __addDamageCuts(self, df):

        percentageDamage = {}

        dpsDF = df[df.Role == "DPS"]
        totalDamage = dpsDF["Damage Done"].sum()
        for player in dpsDF.index:
            percentageDamage[player] = float(dpsDF[dpsDF.index.isin([player])]["Damage Done"] / totalDamage)

        df["Percentage Damage"] = pd.Series(percentageDamage)
        df["Percentage Damage"] = df["Percentage Damage"].fillna(0)
        
        cutDueToDamage = {}
        for player, percent in percentageDamage.items():
            cutDueToDamage[player] = percent * self.dpsCutTotal

        playerMaxDMG = max(percentageDamage, key=percentageDamage.get)
        playerMinDMG = self.__minOfDict(percentageDamage)
        
        percentageDamage[playerMaxDMG] = percentageDamage[playerMaxDMG] + 0.005
        percentageDamage[playerMinDMG] = percentageDamage[playerMinDMG] - 0.005

        damageCut = {}

        for player in dpsDF.index:
            damageCut[player] = percentageDamage[player] * self.dpsCutTotal

        df["Damage Cut"] = pd.Series(damageCut)
        df["Damage Cut"] = df["Damage Cut"].fillna(0)
    
        return df
    
    def __addTankCuts(self, df):
    
        tankCut = {}
        for player in self.tanks:
            tankCut[player] = self.tankCutTotal/ len(self.tanks)

        df["Tank Cut"] = pd.Series(tankCut)
        df["Tank Cut"] = df["Tank Cut"].fillna(0)

        return df
    
    def __addHealerCuts(self, df):
        
        healerCut = {}
        for player in self.healers:
            healerCut[player] = self.healerCutTotal/ len(self.healers)

        df["Healer Cut"] = pd.Series(healerCut)
        df["Healer Cut"] = df["Healer Cut"].fillna(0)

        return df
    
    def __addBonusCuts(self, df):
        
        bonusCut = {}
        
        for player, value in self.bonusCutsDict.items():
            bonusCut[player] = value * self.raiderPot 
        df["Bonus Cut"] = pd.Series(bonusCut)
        df["Bonus Cut"] = df["Bonus Cut"].fillna(0)

        return df
    
    def __addSupportCuts(self, df):
        
        supportSpecs = list(self.supportsCutDict.keys())
        supportDF = df[df.Spec.isin(supportSpecs)]

        supportsCut = {}

        for index, row in supportDF.iterrows():
            supportsCut[index] = self.supportsCutDict[row.Spec] * self.raiderPot

        df["Support Cut"] = pd.Series(supportsCut)
        df["Support Cut"] = df["Support Cut"].fillna(0)
        
        return df
    
    def __addDeductions(self, df):
        
        deductions = {}
        
        for player, amount in self.deductionsCutsDict.items():
            deductions[player] = int(amount) * -1
         
        df["Deductions"] = pd.Series(deductions, dtype = "float64")
        df["Deductions"] = df["Deductions"].fillna(pd.Series(deductions.values(), dtype = "float64").sum()/(len(self.df)-len(self.deductionsCutsDict)) * -1)
        
        return df
    
    def __minOfDict(self, dictionary):
              
        val = min(filter(None, dictionary.values()))
        res = next(k for k, v in dictionary.items() if v == val)  # 2

        return res
    
    def printDFClean(self):

        df = self.df

        fig = go.Figure(data=[go.Table(
            header=dict(values=list(["Name", " Cut"]),
                        fill_color='paleturquoise',
                        align='center'),
            columnwidth = [80,80],
            cells=dict(values=[list(fatMonies.df.index), fatMonies.df["Total Cut"]],
                       fill_color='lavender',
                       align='center'))
        ])
        
        fig.update_layout(xaxis=dict(showgrid=False),
              yaxis=dict(showgrid=False)
        )
        fig.update_layout(

            height=750,
            paper_bgcolor="white"
        )
        
        py_offline.iplot(fig)
    


In [68]:
fatMonies = GDKPCalcuator(
    
                      warcraftLogURL = "https://tbc.warcraftlogs.com/reports/PCkcgzN6r7XqMF82", 
                      pot = 62050, 
                      management = 0.15,
                      tankCut = 0.1375, 
                      healerCut = 0.2975, 
                      noOfTanks = 3,
                      noOfHealers = 7,
                      namesToRemove = ["Blindman","Jshadylock"],
                      bonusCutsDict = {
                                      "Streaming" : 0.003,
                                      "Gezz" : 0.003, 
                                      
                                      },
                      deductionsCutsDict = {
                                            
                                           }
                    )


fatMonies.printDFClean()

gc = gspread.service_account(filename='gdkp-performance-2069da9ef7b9.json')
sh = gc.open("LR GDKP")
worksheet = sh.worksheet("Python API")
df = fatMonies.df.sort_values(by=['Role'],ascending=False).reset_index(level=0).fillna(0)
worksheet.clear()
worksheet.update([df.columns.values.tolist()] + df.values.tolist())

{'spreadsheetId': '1Wb0BMdjrFN6vkcAnvIP9dlVomUL4YM-qK2bL96vIACQ',
 'updatedRange': "'Python API'!A1:O26",
 'updatedRows': 26,
 'updatedColumns': 15,
 'updatedCells': 390}

In [42]:
#print entire data frame

fatMonies.df




,Class,Spec,Damage Done,Healing Done,Damage Taken,Role,Percentage Damage,Damage Cut,Tank Cut,Healer Cut,Support Cut,Bonus Cut,Deductions,Total Cut
Name,,,,,,,,,,,,,,
Bambambamm,Shaman,Shaman-Enhancement,5100918.0,17992,737782,DPS,0.062909,1620.845070,0.000000,0.00000,659.28125,0.0000,-0.0,2281
Blindman,Unknown,Unknown,NaN,153,153,DPS,0.000000,0.000000,0.000000,0.00000,0.00000,0.0000,-0.0,0
Careful,Rogue,Rogue-Combat,6116333.0,61115,720226,DPS,0.075433,1943.498835,0.000000,0.00000,0.00000,0.0000,-0.0,1944
Clutchxd,Druid,Druid-Restoration,NaN,3175866,720724,Healer,0.000000,0.000000,0.000000,2241.55625,0.00000,0.0000,-0.0,2242
Darmar,Shaman,Shaman-Elemental,5303996.0,19132,655696,DPS,0.065414,1685.374234,0.000000,0.00000,659.28125,0.0000,-0.0,2345
Elodié,Paladin,Paladin-Holy,2011.0,1924975,730596,Healer,0.000000,0.000000,0.000000,2241.55625,0.00000,0.0000,-0.0,2242
Fakemechanic,Shaman,Shaman-Restoration,43388.0,3074179,573931,Healer,0.000000,0.000000,0.000000,2241.55625,0.00000,0.0000,-0.0,2242
Gezz,Warlock,Warlock-Destruction,7541233.0,117864,623601,DPS,0.093006,2525.092292,0.000000,0.00000,0.00000,158.2275,-0.0,2684
Grízzly,Druid,Druid-Balance,3840241.0,24720,690972,DPS,0.047362,1220.257940,0.000000,0.00000,659.28125,0.0000,-0.0,1880


In [38]:
#Run this code for diagnostics information:

fatMonies.diagnostics()  


Tank Cut    :  7252.094 
Tank Cut DF :  7252.094


Healer Cut    :  15690.894 
Healer Cut DF :  15690.894


DPS Cut    :  25764.711 
DPS Cut DF :  25764.711


Support Cut    :  3560.119 
Support Cut DF :  3560.119


Bonus Cut    :  474.683 
Bonus Cut DF :  316.455



Total Pot      :  62050
Management Cut :  9307.5
Raider Cut     :  52742.5

Tank Cut    :  7252.094
Healer Cut  :  15690.894
DPS Cut     :  25764.711
Support Cut :  3560.119
Bonus Cut   :  474.683
Total Cuts  :  52742.5


In [39]:
import gspread


gc = gspread.service_account(filename='gdkp-performance-2069da9ef7b9.json')
sh = gc.open("LR GDKP")

{'spreadsheetId': '1Wb0BMdjrFN6vkcAnvIP9dlVomUL4YM-qK2bL96vIACQ',
 'updatedRange': "'Python API'!A1:O26",
 'updatedRows': 26,
 'updatedColumns': 15,
 'updatedCells': 390}